In [ ]:
!pip3 install scholarly

In [ ]:
import pandas as pd
from scholarly import scholarly
from collections import defaultdict
import re
import pickle
import numpy as np
import ast
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/My\ Drive/Coursework/Sem2/ML/PaperAcceptanceMaster/scrape_gs/

/content/drive/My Drive/Coursework/Sem2/ML/PaperAcceptanceMaster/scrape_gs




---



**Set year for scrapping**

In [ ]:
global_year = 2020
global_data_file = "author_info_scholar_2020.pickle"

**RUN ONLY ONCE TO MAKE THE MASTER DATA**

In [ ]:
data_raw = pd.read_pickle('all_data_features_17_20.pkl')
data = data_raw[["id", "title", "label", "authors"]]
org_papers = data.to_dict('index')

In [ ]:
yearwise_authors = {y: set() for y in range(2017, 2021)}

for p in org_papers:
    year = int(p.split("_")[0])
    yearwise_authors[year].add(org_papers[p]["authors"][-1])

In [ ]:
print(len(yearwise_authors[global_year]))

1631


In [ ]:
# Read the global file containing authors already scrapped till now

with open(global_data_file, "rb") as f:
    global_author_info = pickle.load(f)

global_author_info = ast.literal_eval(global_author_info)

print("Done already: ", len(global_author_info))

Done already:  306


In [ ]:
# Remove authors which are already scrapped
author_names_set = yearwise_authors[global_year].difference(set(global_author_info.keys()))
print(len(author_names_set))

1507


In [ ]:
# Split authors list into 4 parts
author_names_set = list(author_names_set)
size_of_each = math.ceil(len(author_names_set)/4)

with open("author_names_to_scrape_0", "w") as f:
    f.write("\n".join(x for x in author_names_set[0: size_of_each]))

with open("author_names_to_scrape_1", "w") as f:
    f.write("\n".join(x for x in author_names_set[size_of_each:2*size_of_each]))

with open("author_names_to_scrape_2", "w") as f:
    f.write("\n".join(x for x in author_names_set[2*size_of_each: 3*size_of_each]))

with open("author_names_to_scrape_3", "w") as f:
    f.write("\n".join(x for x in author_names_set[3*size_of_each:]))


# for idx, i in enumerate(range(0, num_batches)):
#      with open("author_names_to_scrape_{}".format(idx), "w"):
#          f.write("\n".join(x for x in author_names_set[i*size_of_each: (i+1)*size_of_each]))



---



---



**SET SCRIPT NUMBER (same as file name suffix)**

In [ ]:
script_number = 0

In [ ]:
script_num_dict = {}
script_num_file_DATA = "PARTIAL_author_info_scholar_2020_{}.pickle".format(script_number)

In [ ]:
script_author_name_file = "author_names_to_scrape_{}".format(script_number)



---



---



In [ ]:
# RUN THIS ONLY THE FIRST TIME
author_info = {}
error_names = []

count =0
with open(script_num_file_DATA, "wb") as f:
    pickle.dump(author_info.__str__(), f)

In [ ]:
# RESTART FROM HERE AFTER COLAB DISCONNECTS

with open(script_num_file_DATA, "rb") as f:
    author_info = pickle.load(f)

author_info = ast.literal_eval(author_info)
count = len(author_info)

print(count)

EOFError: ignored

In [ ]:
with open(script_author_name_file, "r") as f:
    author_names_for_this_run = [x.strip("\n") for x in f.readlines()]
print(len(author_names_for_this_run), author_names_for_this_run[0:2])

377 ['Dinesh Manocha', 'Bülent Yener']


In [ ]:
p = re.compile(" [A-Z]\.? ")

In [ ]:
for a in author_names_for_this_run:
    if not a in author_info and not a in error_names:
        if count % 10 == 0:
            print("Status {} / {}".format(count, len(author_names_for_this_run)))
            with open(script_num_file_DATA, "wb") as f:
                pickle.dump(author_info.__str__(), f)
        auth_candidates = []
        try:
            c = scholarly.search_author(a)
            fill_count = 0
            for i in c:
                if fill_count > 6:
                    break
                auth_candidates.append((i, i.fill()))
                fill_count += 1

            if not auth_candidates:
                # no author found
                a_clean = p.sub(" ", a)
                c = scholarly.search_author(a_clean)
                fill_count += 1
                for i in c:
                    if fill_count > 6:
                        break
                    auth_candidates.append((i, i.fill()))
                    fill_count += 1

            author_info[a] = auth_candidates
        except Exception as ex:
            print("Exception: ", a)
            print(ex)
            error_names.append(a)
        count += 1

Status 0 / 377
Status 10 / 377
Status 20 / 377
Status 30 / 377
Status 40 / 377
Status 50 / 377
Status 60 / 377
Status 70 / 377
Status 80 / 377
Status 90 / 377
Status 100 / 377
Status 110 / 377
Status 120 / 377
Status 130 / 377
Status 140 / 377
Status 150 / 377
Status 160 / 377
Status 170 / 377
Status 180 / 377
Status 190 / 377
Status 200 / 377
Status 210 / 377
Status 220 / 377
Status 230 / 377
Status 240 / 377
Status 250 / 377
Status 260 / 377
Status 270 / 377
Status 280 / 377
Status 290 / 377
Status 300 / 377
Status 310 / 377
Status 320 / 377
Status 330 / 377
Status 340 / 377
Status 350 / 377
Status 360 / 377
Status 370 / 377


In [ ]:
with open(script_num_file_DATA, "wb") as f:
    pickle.dump(author_info.__str__(), f)

In [ ]:
print(len(author_info))

377


In [ ]:
a

'Marco Pavone'

In [ ]:
len(auth_candidates)

1

In [ ]:
auth_candidates[0]

In [ ]:
author_info["Chen Chen"] = auth_candidates
count += 1

In [ ]:
with open(script_num_file_DATA, "rb") as f:
    from_file_author_info = pickle.load(f)

from_file_author_info = ast.literal_eval(author_info)
print(len(from_file_author_info), len(author_info))